<a href="https://colab.research.google.com/github/noumantechie/langchain/blob/main/chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using Retrieval base chain
**bold text**

In [ ]:
!pip install -qU langchain_community langchain_text_splitters langchain_google_genai langchain_huggingface faiss-cpu


In [3]:
# Import necessary modules
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate

# Corrected FAISS import
from langchain_community.vectorstores import FAISS

# Import necessary chain functions
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import os



In [5]:
URL1 = "https://techcrunch.com/2024/03/04/anthropic-claims-its-new-models-beat-gpt-4/"
URL2 = "https://techcrunch.com/2024/03/28/ai21-labs-new-text-generating-ai-model-is-more-efficient-than-most/"

In [6]:
loader = WebBaseLoader([URL1, URL2])
data = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=200,
    chunk_overlap=50,
)
chunks = text_splitter.split_documents(data)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [11]:
vectorstore = FAISS.from_documents(chunks , embeddings)
retriever = vectorstore.as_retriever()


In [12]:
prompt_template = """
Answer the question "{input}" based solely on the context below:

<context>
{context}
</context>

If you can't find an answer, say "I don't know."
"""


In [14]:
prompt = PromptTemplate.from_template(prompt_template)

In [13]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=GOOGLE_API_KEY  # Corrected 'api_ket' to 'api_key'
)

**Using Create stuff chain**

In [15]:
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

**Using Retrieval chain**

In [18]:
chain = create_retrieval_chain(retriever, combine_docs_chain)

In [ ]:
result = chain.invoke({"input": "Liast the models and their token size of models only from Anthropic and Meta"})


In [25]:
print(result)


{'input': 'Liast the models and their token size of models only from Anthropic and Meta', 'context': [Document(id='e3fe54db-edcd-441d-960d-b297e63af18b', metadata={'source': 'https://techcrunch.com/2024/03/04/anthropic-claims-its-new-models-beat-gpt-4/', 'title': "Anthropic claims its new AI chatbot models beat OpenAI's GPT-4 | TechCrunch", 'description': 'Anthropic, the AI startup funded backed by Google, Amazon and others, has released its latest flagship GenAI models, Claude 3.', 'language': 'en-US'}, page_content='having models respond to questions and perform tasks using a simple set of guiding principles. For example, for Claude 3, Anthropic said that it added a principle — informed by crowdsourced feedback'), Document(id='5cbbaa8d-8b47-443c-a4f0-6f81e7087eee', metadata={'source': 'https://techcrunch.com/2024/03/28/ai21-labs-new-text-generating-ai-model-is-more-efficient-than-most/', 'title': "AI21 Labs' new AI model can handle more context than most | TechCrunch", 'description':